In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

In [27]:
# !pip install xgboost


In [2]:
df = pd.read_csv("data_processed.csv")
df

,Giá,Năm sản xuất,Kiểu dáng,Xuất xứ,Số km đã đi,Hộp số,Nhiên liệu
0,1.119000e+09,2016,Sedan,Trongnước,51.499,Sốtựđộng,Xăng
1,1.789000e+09,2021,SUV,Nhậpkhẩu,8.000,Sốtựđộng,Xăng
2,5.200000e+08,2021,MPV,Nhậpkhẩu,10.000,Sốtay,Xăng
3,3.200000e+08,2009,Sedan,Trongnước,90.000,Sốtựđộng,Xăng
4,7.300000e+08,2016,SUV,Trongnước,68.000,Sốtựđộng,Xăng
...,...,...,...,...,...,...,...
2291,6.420000e+08,2018,Pick-upTruck,Nhậpkhẩu,15.000,Sốtựđộng,Diesel
2292,8.980000e+08,2018,Sedan,Trongnước,50.000,Sốtựđộng,Xăng
2293,3.800000e+08,2014,Hatchback,Nhậpkhẩu,92.000,Sốtựđộng,Xăng
2294,1.270000e+08,1993,Sedan,Nhậpkhẩu,68.000,Sốtựđộng,Xăng


## Sử dụng Linear Regression,Random Forest,Gradient Boosting

# Linear Regression

In [9]:
 # Đọc file data
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float) 

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=60
)

# Huấn luyện mô hình
model = LinearRegression()
model.fit(X_train, y_train)

# Dự đoán giá xe với thông tin mới
new_car_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "Sedan",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuyển đổi dữ liệu mới sang dạng số
new_car_data_encoded = pd.DataFrame([new_car_data], columns=new_car_data.keys())
encoded_data = encoder.transform(new_car_data_encoded[categorical_cols]).toarray()  
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))  # Tạo DataFrame từ mảng NumPy


y_pred = model.predict(X_test)

# Tính các chỉ số đánh giá
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)

new_car_data_encoded = new_car_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)  # Kết hợp DataFrame

# Dự đoán giá xe
predicted_price = model.predict(new_car_data_encoded)
formatted_price = format(predicted_price[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price,"VND")


Mean Absolute Error (MAE): 591140337.724638
Mean Squared Error (MSE): 9.167266394475543e+17
R² Score: 0.2633008020622033
Độ chính xác của mô hình: 26.33%
Giá xe dự đoán: 1,090,725,848 VND


# Ridge Regression

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Khởi tạo mô hình Ridge Regression
model = Ridge(alpha=1.0)  # Alpha là hyperparameter điều chỉnh độ phức tạp

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Dự đoán giá trị cho tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá hiệu suất của mô hình
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

Mean Squared Error: 2.232132593422841e+18
Root Mean Squared Error: 1494032326.766339
R-squared: 0.20045134909215012


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=60
)

# Định nghĩa khoảng giá trị cho alpha
alpha_values = {'alpha': [0.001, 0.1, 1.0, 10.0, 200.0]}

# Khởi tạo mô hình Ridge Regression
ridge = Ridge()

# Khởi tạo Grid Search với cross-validation
grid_search = GridSearchCV(estimator=ridge, param_grid=alpha_values, scoring='r2', cv=5)

# Huấn luyện mô hình trên tập huấn luyện
grid_search.fit(X_train, y_train)

# Lấy giá trị alpha tốt nhất
best_alpha = grid_search.best_params_['alpha']
print(f"Best alpha: {best_alpha}")

# Khởi tạo mô hình với alpha tốt nhất
optimized_model = Ridge(alpha=best_alpha)

# Huấn luyện mô hình với alpha tốt nhất
optimized_model.fit(X_train, y_train)

# Dự đoán giá trị cho tập kiểm tra
optimized_y_pred = optimized_model.predict(X_test)

# Đánh giá hiệu suất của mô hình
optimized_mse = mean_squared_error(y_test, optimized_y_pred)
optimized_rmse = mean_squared_error(y_test, optimized_y_pred, squared=False)
optimized_r2 = r2_score(y_test, optimized_y_pred)

print(f"Optimized Mean Squared Error: {optimized_mse}")
print(f"Optimized Root Mean Squared Error: {optimized_rmse}")
print(f"Optimized R-squared: {optimized_r2}")


Best alpha: 200.0
Optimized Mean Squared Error: 9.293863914989919e+17
Optimized Root Mean Squared Error: 964046882.4175471
Optimized R-squared: 0.2531271812890513


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Định nghĩa khoảng giá trị cho alpha
alpha_values = {'alpha': np.logspace(-3, 3, 50)}  # Giá trị alpha từ 0.001 đến 1000

# Khởi tạo mô hình Ridge Regression
ridge = Ridge()

# Khởi tạo Randomized Search với cross-validation
random_search = RandomizedSearchCV(estimator=ridge, param_distributions=alpha_values, scoring='r2', cv=5, n_iter=50, random_state=42)

# Huấn luyện mô hình trên tập huấn luyện
random_search.fit(X_train, y_train)

# Lấy giá trị alpha tốt nhất
best_alpha = random_search.best_params_['alpha']
print(f"Best alpha: {best_alpha}")

# Khởi tạo mô hình với alpha tốt nhất
optimized_model = Ridge(alpha=best_alpha)

# Huấn luyện mô hình với alpha tốt nhất
optimized_model.fit(X_train, y_train)

# Dự đoán giá trị cho tập kiểm tra
optimized_y_pred = optimized_model.predict(X_test)

# Đánh giá hiệu suất của mô hình
optimized_mse = mean_squared_error(y_test, optimized_y_pred)
optimized_rmse = mean_squared_error(y_test, optimized_y_pred, squared=False)
optimized_r2 = r2_score(y_test, optimized_y_pred)

print(f"Optimized Mean Squared Error: {optimized_mse}")
print(f"Optimized Root Mean Squared Error: {optimized_rmse}")
print(f"Optimized R-squared: {optimized_r2}")


Best alpha: 1.5264179671752334
Optimized Mean Squared Error: 2.23050305946113e+18
Optimized Root Mean Squared Error: 1493486879.5744843
Optimized R-squared: 0.20103504724903098


# Tổng quan về Linear Regression, Ridge Regression và GridSearchCV, RandomizedSearchCV
Optimized Mean Squared Error (MSE): 2.23050305946113e+18: Giá trị MSE rất cao, cho thấy mô hình có độ lỗi rất lớn khi dự đoán giá trị giá cả.

Optimized Root Mean Squared Error (RMSE): 1493486879.5744843: Giá trị RMSE cũng rất cao, đồng nghĩa với việc mô hình không dự đoán chính xác giá trị thực tế.

Optimized R-squared: 0.20103504724903098: Giá trị R-squared thấp, chỉ ra rằng mô hình chỉ giải thích được khoảng 20% biến thiên của dữ liệu, còn lại 80% biến thiên không được giải thích bởi mô hình.

# Gradient Boosting

In [4]:
 from sklearn.ensemble import GradientBoostingRegressor
 # Đọc file data
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=0
)

# Huấn luyện mô hình
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

# Dự đoán giá xe với thông tin mới
new_car_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuyển đổi dữ liệu mới sang dạng số
new_car_data_encoded = pd.DataFrame([new_car_data], columns=new_car_data.keys())
encoded_data = encoder.transform(new_car_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))  # Tạo DataFrame từ mảng NumPy


y_pred = model.predict(X_test)

# Tính các chỉ số đánh giá
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)

new_car_data_encoded = new_car_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)  # Kết hợp DataFrame

# Dự đoán giá xe
predicted_price = model.predict(new_car_data_encoded)
formatted_price = format(predicted_price[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price,"VND")


Mean Absolute Error (MAE): 446646686.96598685
Mean Squared Error (MSE): 1.1209545757865522e+18
R² Score: 0.4545526020712196
Độ chính xác của mô hình: 45.46%
Giá xe dự đoán: 1,141,295,004 VND


# Random Forest

In [5]:
 from sklearn.ensemble import RandomForestRegressor
 # Đọc file data
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.1, random_state=42
)

# Huấn luyện mô hình
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Dự đoán giá xe với thông tin mới
new_car_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuyển đổi dữ liệu mới sang dạng số
new_car_data_encoded = pd.DataFrame([new_car_data], columns=new_car_data.keys())
encoded_data = encoder.transform(new_car_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))  # Tạo DataFrame từ mảng NumPy


y_pred = model.predict(X_test)

# Tính các chỉ số đánh giá
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)

new_car_data_encoded = new_car_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)  # Kết hợp DataFrame

# Dự đoán giá xe
predicted_price = model.predict(new_car_data_encoded)
formatted_price = format(predicted_price[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price,"VND")


Mean Absolute Error (MAE): 412375935.8703904
Mean Squared Error (MSE): 2.1723142103527736e+18
R² Score: 0.4472344438391529
Độ chính xác của mô hình: 44.72%
Giá xe dự đoán: 1,131,748,231 VND


# Sử dụng tham số GridSearchCV cho RandomForest

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.4, random_state=60
)

# Sử dụng GridSearchCV để tìm kiếm các tham số tốt nhất cho mô hình Random Forest
param_grid = {
    'n_estimators': [20,50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
rf_model = RandomForestRegressor()
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Lấy mô hình tốt nhất đã được tìm kiếm
best_rf_model = grid_search.best_estimator_

# Dự đoán giá trị trên tập kiểm tra
y_pred_best = best_rf_model.predict(X_test)

# Đánh giá hiệu suất của mô hình tốt nhất
r2_best = r2_score(y_test, y_pred_best)
mae_best = mean_absolute_error(y_test, y_pred_best)
mse_best = mean_squared_error(y_test, y_pred_best)

print("Random Forest (best) - R² Score:", r2_best)
print("Random Forest (best) - Mean Absolute Error (MAE):", mae_best)
print("Random Forest (best) - Mean Squared Error (MSE):", mse_best)

# Dữ liệu mới cần dự đoán
new_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuẩn bị dữ liệu mới
new_data_encoded = pd.DataFrame([new_data])
new_data_encoded["Số km đã đi"] = new_data_encoded["Số km đã đi"].astype(float)
encoded_data = encoder.transform(new_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_data_encoded = new_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)
new_data_encoded[numerical_cols] = scaler.transform(new_data_encoded[numerical_cols])

# Dự đoán giá trị của dữ liệu mới
predicted_price_best = best_rf_model.predict(new_data_encoded)

formatted_price_best = format(predicted_price_best[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price,"VND")


Random Forest (best) - R² Score: 0.3051047534689929
Random Forest (best) - Mean Absolute Error (MAE): 397980076.81809276
Random Forest (best) - Mean Squared Error (MSE): 1.0852121434584617e+18
Giá xe dự đoán: 1,131,748,231 VND


# Sử dụng Random Forest Regression với RandomizedSearchCV để tối ưu hóa hyperparameter

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Định nghĩa các giá trị hyperparameter cần thử nghiệm
param_distributions = {
    'rf__n_estimators': [50, 100, 200, 300, 400],
    'rf__max_depth': [5, 10, 15, 20, 25],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__max_features': ['sqrt', 'log2']
}

# Sử dụng Pipeline để kết hợp các bước tiền xử lý và huấn luyện
pipeline = Pipeline([
    ('rf', RandomForestRegressor())
])

# Khởi tạo RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Huấn luyện RandomizedSearchCV với dữ liệu train
random_search.fit(X_train, y_train)

# Lấy tổ hợp hyperparameter tốt nhất
best_params = random_search.best_params_
print(f"Best hyperparameters: {best_params}")

# Tạo mô hình mới với hyperparameter tốt nhất
best_model = random_search.best_estimator_

# Dự đoán giá trị cho tập kiểm tra
y_pred = best_model.predict(X_test)

# Đánh giá hiệu suất của mô hình tốt nhất
r2_best = r2_score(y_test, y_pred)
mae_best = mean_absolute_error(y_test, y_pred)
mse_best = mean_squared_error(y_test, y_pred)

print("Random Forest (best) - R² Score:", r2_best)
print("Random Forest (best) - Mean Absolute Error (MAE):", mae_best)
print("Random Forest (best) - Mean Squared Error (MSE):", mse_best)

# Dữ liệu mới cần dự đoán
new_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuẩn bị dữ liệu mới
new_data_encoded = pd.DataFrame([new_data])
new_data_encoded["Số km đã đi"] = new_data_encoded["Số km đã đi"].astype(float)
encoded_data = encoder.transform(new_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_data_encoded = new_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)
new_data_encoded[numerical_cols] = scaler.transform(new_data_encoded[numerical_cols])

# Dự đoán giá trị của dữ liệu mới
predicted_price_best = best_model.predict(new_data_encoded)

formatted_price_best = format(predicted_price_best[0], ",.0f") 
print("Giá xe dự đoán:", formatted_price_best, "VND")


Best hyperparameters: {'rf__n_estimators': 200, 'rf__min_samples_split': 2, 'rf__min_samples_leaf': 1, 'rf__max_features': 'log2', 'rf__max_depth': 25}
Random Forest (best) - R² Score: 0.4124299056734214
Random Forest (best) - Mean Absolute Error (MAE): 380533503.1342051
Random Forest (best) - Mean Squared Error (MSE): 1.6403434074683308e+18
Giá xe dự đoán: 1,251,974,898 VND


# Sử dụng XGBRegressor

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Khởi tạo mô hình XGBoost
model = XGBRegressor()

# Định nghĩa các giá trị hyperparameter cần thử nghiệm
param_distributions = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Khởi tạo RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=50,  # Tăng số lần thử nghiệm
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

# Huấn luyện RandomizedSearchCV với dữ liệu train
random_search.fit(X_train, y_train)

# Lấy tổ hợp hyperparameter tốt nhất
best_params = random_search.best_params_
print(f"Best hyperparameters: {best_params}")

# Tạo mô hình mới với hyperparameter tốt nhất
best_model = random_search.best_estimator_

# Dự đoán giá trị cho tập kiểm tra
y_pred = best_model.predict(X_test)

# Đánh giá hiệu suất của mô hình tốt nhất
r2_best = r2_score(y_test, y_pred)
mae_best = mean_absolute_error(y_test, y_pred)
mse_best = mean_squared_error(y_test, y_pred)

print("XGBoost (best) - R² Score:", r2_best)
print("XGBoost (best) - Mean Absolute Error (MAE):", mae_best)
print("XGBoost (best) - Mean Squared Error (MSE):", mse_best)

# Dữ liệu mới cần dự đoán
new_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuẩn bị dữ liệu mới
new_data_encoded = pd.DataFrame([new_data])
new_data_encoded["Số km đã đi"] = new_data_encoded["Số km đã đi"].astype(float)
encoded_data = encoder.transform(new_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_data_encoded = new_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)
new_data_encoded[numerical_cols] = scaler.transform(new_data_encoded[numerical_cols])

# Dự đoán giá trị của dữ liệu mới
predicted_price_best = best_model.predict(new_data_encoded)

formatted_price_best = format(predicted_price_best[0], ",.0f") 
print("Giá xe dự đoán:", formatted_price_best, "VND")


Best hyperparameters: {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.05, 'colsample_bytree': 0.8}
XGBoost (best) - R² Score: 0.38980152085815156
XGBoost (best) - Mean Absolute Error (MAE): 430142834.4347826
XGBoost (best) - Mean Squared Error (MSE): 1.7035159926829102e+18
Giá xe dự đoán: 1,194,058,240 VND


# Mô hình RandomForest

# sử dụng Gradient Boosting Regression với RandomizedSearchCV để tối ưu hóa hyperparameter

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Khởi tạo mô hình Gradient Boosting Regression
model = GradientBoostingRegressor()

# Định nghĩa các giá trị hyperparameter cần thử nghiệm 
param_distributions = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.05],
    'subsample': [0.6, 0.8] 
}

# Khởi tạo RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=10,  # Số lần lặp
    cv=7,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Huấn luyện RandomizedSearchCV với dữ liệu train
random_search.fit(X_train, y_train)

# Lấy tổ hợp hyperparameter tốt nhất
best_params = random_search.best_params_

# Tạo mô hình mới với hyperparameter tốt nhất
best_model = GradientBoostingRegressor(**best_params)

# Huấn luyện mô hình với hyperparameter tốt nhất
best_model.fit(X_train, y_train)

# Dự đoán giá trị cho tập kiểm tra
y_pred = best_model.predict(X_test)

# Đánh giá hiệu suất của mô hình tốt nhất
r2_best = r2_score(y_test, y_pred)
mae_best = mean_absolute_error(y_test, y_pred)
mse_best = mean_squared_error(y_test, y_pred)

print("Gradient Boosting (best) - R² Score:", r2_best)
print("Gradient Boosting (best) - Mean Absolute Error (MAE):", mae_best)
print("Gradient Boosting (best) - Mean Squared Error (MSE):", mse_best)

# Dữ liệu mới cần dự đoán
new_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuẩn bị dữ liệu mới
new_data_encoded = pd.DataFrame([new_data])
new_data_encoded["Số km đã đi"] = new_data_encoded["Số km đã đi"].astype(float)
encoded_data = encoder.transform(new_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_data_encoded = new_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)
new_data_encoded[numerical_cols] = scaler.transform(new_data_encoded[numerical_cols])

# Dự đoán giá trị của dữ liệu mới
predicted_price_best = best_model.predict(new_data_encoded)

formatted_price_best = format(predicted_price_best[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price_best,"VND")

Gradient Boosting (best) - R² Score: 0.3587424116608007
Gradient Boosting (best) - Mean Absolute Error (MAE): 450059105.7278426
Gradient Boosting (best) - Mean Squared Error (MSE): 1.7902249751611715e+18
Giá xe dự đoán: 1,604,105,839 VND


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Sử dụng RandomizedSearchCV để tìm kiếm các tham số tốt nhất cho mô hình Random Forest
param_dist = {
    'n_estimators': [20, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
rf_model = RandomForestRegressor()
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, n_iter=20, cv=5, scoring='r2', random_state=42)
random_search.fit(X_train, y_train)

# Lấy mô hình tốt nhất đã được tìm kiếm
best_rf_model = random_search.best_estimator_

# Dự đoán giá trị trên tập kiểm tra
y_pred_best = best_rf_model.predict(X_test)

# Đánh giá hiệu suất của mô hình tốt nhất
r2_best = r2_score(y_test, y_pred_best)
mae_best = mean_absolute_error(y_test, y_pred_best)
mse_best = mean_squared_error(y_test, y_pred_best)

print("Random Forest (best) - R² Score:", r2_best)
print("Random Forest (best) - Mean Absolute Error (MAE):", mae_best)
print("Random Forest (best) - Mean Squared Error (MSE):", mse_best)

# Dữ liệu mới cần dự đoán
new_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuẩn bị dữ liệu mới
new_data_encoded = pd.DataFrame([new_data])
new_data_encoded["Số km đã đi"] = new_data_encoded["Số km đã đi"].astype(float)
encoded_data = encoder.transform(new_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_data_encoded = new_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)
new_data_encoded[numerical_cols] = scaler.transform(new_data_encoded[numerical_cols])

# Dự đoán giá trị của dữ liệu mới
predicted_price_best = best_rf_model.predict(new_data_encoded)

formatted_price_best = format(predicted_price_best[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price_best,"VND")


Random Forest (best) - R² Score: 0.3575921950855404
Random Forest (best) - Mean Absolute Error (MAE): 401691036.4107548
Random Forest (best) - Mean Squared Error (MSE): 1.7934360817076193e+18
Độ chính xác của mô hình: 26.33%
Giá xe dự đoán: 1,485,856,756 VND


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")
# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")
# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)
# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)
# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

#khởi tạo mô hình
model = RandomForestRegressor()
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
}

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Khởi tạo Grid Search
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')

# Huấn luyện Grid Search với dữ liệu train
grid_search.fit(X_train, y_train)

# Lấy tổ hợp hyperparameter tốt nhất
best_params = grid_search.best_params_

# Tạo mô hình mới với hyperparameter tốt nhất
best_model = RandomForestRegressor(**best_params)

# Huấn luyện mô hình với hyperparameter tốt nhất
best_model.fit(X_train, y_train)

# Dự đoán giá trị cho tập test
y_pred = best_model.predict(X_test)

# Đánh giá hiệu suất của mô hình tốt nhất
r2_best = r2_score(y_test, y_pred_best)
mae_best = mean_absolute_error(y_test, y_pred_best)
mse_best = mean_squared_error(y_test, y_pred_best)

print("Random Forest (best) - R² Score:", r2_best)
print("Random Forest (best) - Mean Absolute Error (MAE):", mae_best)
print("Random Forest (best) - Mean Squared Error (MSE):", mse_best)

# Dữ liệu mới cần dự đoán
new_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuẩn bị dữ liệu mới
new_data_encoded = pd.DataFrame([new_data])
new_data_encoded["Số km đã đi"] = new_data_encoded["Số km đã đi"].astype(float)
encoded_data = encoder.transform(new_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_data_encoded = new_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)
new_data_encoded[numerical_cols] = scaler.transform(new_data_encoded[numerical_cols])

# Dự đoán giá trị của dữ liệu mới
predicted_price_best = best_rf_model.predict(new_data_encoded)

formatted_price_best = format(predicted_price_best[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price,"VND")


Random Forest (best) - R² Score: 0.3575921950855404
Random Forest (best) - Mean Absolute Error (MAE): 401691036.4107548
Random Forest (best) - Mean Squared Error (MSE): 1.7934360817076193e+18
Giá xe dự đoán: 1,090,725,848 VND


In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float)

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Khởi tạo mô hình Random Forest
model = RandomForestRegressor()

# Định nghĩa các giá trị hyperparameter cần thử nghiệm 
param_distributions = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_depth': [5, 10, 15, 20, 25],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Khởi tạo RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=10,  # Số lần lặp
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Huấn luyện RandomizedSearchCV với dữ liệu train
random_search.fit(X_train, y_train)

# Lấy tổ hợp hyperparameter tốt nhất
best_params = random_search.best_params_

# Tạo mô hình mới với hyperparameter tốt nhất
best_model = RandomForestRegressor(**best_params)

# Huấn luyện mô hình với hyperparameter tốt nhất
best_model.fit(X_train, y_train)

# Dự đoán giá trị cho tập kiểm tra
y_pred = best_model.predict(X_test)

# Đánh giá hiệu suất của mô hình tốt nhất
r2_best = r2_score(y_test, y_pred)
mae_best = mean_absolute_error(y_test, y_pred)
mse_best = mean_squared_error(y_test, y_pred)

print("Random Forest (best) - R² Score:", r2_best)
print("Random Forest (best) - Mean Absolute Error (MAE):", mae_best)
print("Random Forest (best) - Mean Squared Error (MSE):", mse_best)

# Dữ liệu mới cần dự đoán
new_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuẩn bị dữ liệu mới
new_data_encoded = pd.DataFrame([new_data])
new_data_encoded["Số km đã đi"] = new_data_encoded["Số km đã đi"].astype(float)
encoded_data = encoder.transform(new_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_data_encoded = new_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)
new_data_encoded[numerical_cols] = scaler.transform(new_data_encoded[numerical_cols])

# Dự đoán giá trị của dữ liệu mới
predicted_price_best = best_model.predict(new_data_encoded)

formatted_price_best = format(predicted_price_best[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price_best,"VND")

Random Forest (best) - R² Score: 0.44074824627738685
Random Forest (best) - Mean Absolute Error (MAE): 374002311.0487786
Random Forest (best) - Mean Squared Error (MSE): 1.5612859405062034e+18
Giá xe dự đoán: 1,301,961,657 VND


# Sử dụng RandomForest

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

# Đọc dữ liệu từ file CSV
df = pd.read_csv("data_processed.csv")

# Sửa lỗi chính tả
df["Số km đã đi"] = df["Số km đã đi"].str.replace(".", "")

# Chuyển đổi kiểu dữ liệu
df["Số km đã đi"] = df["Số km đã đi"].astype(float) 

# Chuyển đổi các cột categorical sang dạng số
categorical_cols = ["Kiểu dáng", "Xuất xứ", "Hộp số", "Nhiên liệu"]
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[categorical_cols]).toarray()
encoded_features_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
df = df.drop(categorical_cols, axis=1).join(encoded_features_df)

# Chuẩn hóa các cột số
scaler = StandardScaler()
numerical_cols = ["Năm sản xuất", "Số km đã đi"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("Giá", axis=1), df["Giá"], test_size=0.2, random_state=42
)

# Khởi tạo mô hình Random Forest
model = RandomForestRegressor()

# Định nghĩa các giá trị hyperparameter cần thử nghiệm 
# Sử dụng RandomizedSearchCV để thử nghiệm nhiều giá trị hyperparameter hơn GridSearchCV
param_distributions = {
    'rf__n_estimators': [50, 100, 200, 300, 400],  # Sử dụng 'rf__n_estimators' để chỉ định tham số cho 'RandomForestRegressor'
    'rf__max_depth': [5, 10, 15, 20, 25],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__max_features': ['sqrt', 'log2']
}

# Sử dụng Pipeline để kết hợp các bước tiền xử lý và huấn luyện
pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('rf', RandomForestRegressor()) 
])

# Khởi tạo RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=pipeline, # Sử dụng pipeline thay vì trực tiếp model
    param_distributions=param_distributions,
    n_iter=10,  # Số lần lặp
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Huấn luyện RandomizedSearchCV với dữ liệu train
random_search.fit(X_train, y_train)

# Lấy tổ hợp hyperparameter tốt nhất
best_params = random_search.best_params_

# Tạo mô hình mới với hyperparameter tốt nhất
best_model = RandomForestRegressor(n_estimators=best_params['rf__n_estimators'], # Lấy hyperparameter từ pipeline
                                 max_depth=best_params['rf__max_depth'],
                                 min_samples_split=best_params['rf__min_samples_split'],
                                 min_samples_leaf=best_params['rf__min_samples_leaf'],
                                 max_features=best_params['rf__max_features'])

# Huấn luyện mô hình với hyperparameter tốt nhất
best_model.fit(X_train, y_train)

# Dự đoán giá trị cho tập test
y_pred = best_model.predict(X_test)

# Đánh giá hiệu suất của mô hình tốt nhất
r2_best = r2_score(y_test, y_pred)
mae_best = mean_absolute_error(y_test, y_pred)
mse_best = mean_squared_error(y_test, y_pred)

print("Random Forest (best) - R² Score:", r2_best)
print("Random Forest (best) - Mean Absolute Error (MAE):", mae_best)
print("Random Forest (best) - Mean Squared Error (MSE):", mse_best)

# Dữ liệu mới cần dự đoán
new_data = {
    "Năm sản xuất": 2016,
    "Số km đã đi": '51499',
    "Kiểu dáng": "SUV",
    "Xuất xứ": "Trong nước",
    "Hộp số": "Số tự động",
    "Nhiên liệu": "Xăng"
}

# Chuẩn bị dữ liệu mới
new_data_encoded = pd.DataFrame([new_data])
new_data_encoded["Số km đã đi"] = new_data_encoded["Số km đã đi"].astype(float)
encoded_data = encoder.transform(new_data_encoded[categorical_cols]).toarray()
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))
new_data_encoded = new_data_encoded.drop(categorical_cols, axis=1).join(encoded_df)
new_data_encoded[numerical_cols] = scaler.transform(new_data_encoded[numerical_cols])

# Dự đoán giá trị của dữ liệu mới
predicted_price_best = best_model.predict(new_data_encoded)

formatted_price_best = format(predicted_price_best[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price_best,"VND")

Random Forest (best) - R² Score: 0.4127175331294287
Random Forest (best) - Mean Absolute Error (MAE): 379780793.8535268
Random Forest (best) - Mean Squared Error (MSE): 1.6395404261630804e+18
Giá xe dự đoán: 1,362,073,617 VND


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error


data = pd.read_csv("data_processed1.csv")
# Data preprocessing
# Separate the features and the target variable
X = data.drop(columns='Giá')
y = data['Giá']

# Identify categorical and numerical columns
categorical_cols = ['Năm sản xuất', 'Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']
numerical_cols = ['Năm sản xuất','Số km đã đi']

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)
                          ])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Fit the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 370363384.7164099
Mean Squared Error (MSE): 1.1488283915377326e+18
Root Mean Squared Error (RMSE): 1071834125.0108305
R^2 Score: 0.440989429574993


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error


data = pd.read_csv("data_processed1.csv")

# Split the features and target variable again
X = data.drop(columns='Giá')
y = data['Giá']

# Re-identify the numerical columns now that "Số km đã đi" is correctly formatted
numerical_cols = ['Số km đã đi']

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)
                          ])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Fit the model
pipeline.fit(X_train, y_train)

# Make predictionsa
y_pred = pipeline.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 369147792.3121651
Mean Squared Error (MSE): 1.1528536972059821e+18
Root Mean Squared Error (RMSE): 1073710248.254147
R^2 Score: 0.43903074850972845


In [86]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

data = pd.read_csv("data_processed1.csv")

# Split the features and target variable again
X = data.drop(columns='Giá')
y = data['Giá']

# Re-identify the numerical columns now that "Số km đã đi" is correctly formatted
numerical_cols = ['Số km đã đi']
categorical_cols = ['Năm sản xuất', 'Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=10)

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)
                          ])

# Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=39) #65%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=39    )

# Fit the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 267223339.46358734
Mean Squared Error (MSE): 4.449812423444236e+17
Root Mean Squared Error (RMSE): 667069143.6008891
R^2 Score: 0.654674398460344


In [110]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

data = pd.read_csv("data_processed1.csv")

# Split the features and target variable again
X = data.drop(columns='Giá')
y = data['Giá']

# Re-identify the numerical columns now that "Số km đã đi" is correctly formatted
numerical_cols = ['Số km đã đi']
categorical_cols = ['Năm sản xuất', 'Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=10)

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)
                          ])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, random_state=39    )

# Fit the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 271934041.8294689
Mean Squared Error (MSE): 4.687191410458959e+17
R^2 Score: 0.6955848016413639


Với kết quả này, hãy cùng xem xét và đánh giá hiệu suất của mô hình:

1. **Mean Absolute Error (MAE)**: Giá trị MAE là khoảng 271,934,041.83. Điều này có nghĩa là trung bình, dự đoán của mô hình lệch khoảng 271 triệu so với giá trị thực tế. MAE cung cấp một ý niệm rõ ràng về độ lớn của lỗi trong dự đoán.

2. **Mean Squared Error (MSE)**: MSE có giá trị khoảng 4.687e+17, cho thấy rằng các lỗi của mô hình tập trung ở mức khá cao. Giá trị MSE lớn, đặc biệt khi so với MAE, báo hiệu có sự hiện diện của một số lỗi rất lớn trong dự đoán vì MSE phạt nặng các lỗi l 3ợp.

4. **R² Score**: giá trị là khoảng 0.70, cho thấy rằng khoảng 70% sự biến thiên của dữ liệu giá có thể được giải thích bởi mô hình. Đây là một dấu hiệu tích cực cho thấy mô hình có khả năng học được mối quan hệ giữa dữ liệu đầu vào và giá trị đầu ra. Tuy nhiên, một R² Score không đủ cao cũng cho thấy có chỗ cho việc cải thiện.

Dựa trên những đánh giá trên:

- Có thể hình của bạn đã có khả năng dự đoán một phần dữ liệu tương đối tốt (R² ~0.70). Tuy nhiên, các chỉMSE, và RMSE đều báo hiệu rằng mô hình đang gặp vấn có thể đề với việc dự đoán khôở mức độ lớn, có thể do overfitting hoặc dữ liệu có sự biến thiên rộng lớn mà mô hình chưa thể c quan trọng.
- Khả năng dùng mô hình này cho dự đoán còn tùy thuộc vào yêu cầu cụ thể về độ chính xác và mức độ chấp nhận được của sai số trong ứng dụng thực tế. Nếu mô hình được dùng để dự đoán trong một ngữ cảnh mà sai số 271 triệu không phải vấn đề lớn (giả sử giá trị của mục tiêu rất lớn), thì có lẽ mô hình này vẫn còn đáng để xem xét. Tuy nhiên, nếu mức độ chính xác cần thiết cao, bạn cần cải thiện mô hình.

# Loại bỏ ngoại lệ

In [112]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu
data = pd.read_csv("data_processed1.csv")

# Phát hiện và xử lý ngoại lệ
Q1 = data['Giá'].quantile(0.25)
Q3 = data['Giá'].quantile(0.75)
IQR = Q3 - Q1
filter = (data['Giá'] >= Q1 - 1.5 * IQR) & (data['Giá'] <= Q3 + 1.5 * IQR)
data_filtered = data.loc[filter]

# Tách biến đặc trưng và biến mục tiêu
X = data_filtered.drop(columns='Giá')
y = data_filtered['Giá']

# Re-định lại cột dữ liệu số sau khi xử lý ngoại lệ
numerical_cols = ['Số km đã đi']

# Xác định cột phân loại
categorical_cols = ['Năm sản xuất', 'Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']

# Tiền xử lý cho dữ liệu số
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Tiền xử lý cho dữ liệu phân loại
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Gộp tiền xử lý cho dữ liệu số và phân loại
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Định nghĩa mô hình
model = RandomForestRegressor(random_state=10)

# Tạo pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, random_state=39)

# Huấn luyện mô hình
pipeline.fit(X_train, y_train)

# Dự đoán
y_pred = pipeline.predict(X_test)

# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 86024408.03641494
Mean Squared Error (MSE): 3.0737715258840668e+16
Root Mean Squared Error (RMSE): 175321747.82051617
R^2 Score: 0.7337910718478302


- sử dụng phương pháp IQR để phát hiện và loại bỏ ngoại lệ từ biến mục tiêu "Giá".
- Sau đó, chúng ta tiếp tục với các bước tiền xử lý và mô hình hóa như bình thường, nhưng sử dụng dữ liệu đã được xử lý ngoại lệ.

Kết quả sau khi đã xử lý ngoại lệ cho thấy một sự cải thiện đáng kể trong chất lượng dự đoán so với trước khi xử lý. Dưới đây là so sánh:

### Trước khi xử lý ngoại lệ:
- Mean Absolute Error (MAE): 271934041.8294689
- Mean Squared Error (MSE): 4.687191410458959e+17
- Root Mean Squared Error (RMSE): 684630660.3168572
- R² Score: 0.6955848016413639

### Sau khi xử lý ngoại lệ:
- Mean Absolute Error (MAE): 86024408.03641494
- Mean Squared Error (MSE): 3.0737715258840668e+16
- Root Mean Squared Error (RMSE): 175321747.82051617
- R² Score: 0.733791071802

Như bạn có thể thấy, sau khi xử lý ngoại lệ, cả MAE và MSE đều giảm đáng kể, và R² Score đã tăng. Điều này cho thấy rằng việc loại bỏ ngoại lệ đã làm giảm hiệu ứng của các điểm dữ liệu bất thường đối với mô hình, từ đó cải thiện khả năng dự đoán và chính xác của mô hình.

# Phương pháp thay thế ngoại lệ

In [114]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu
data = pd.read_csv("data_processed1.csv")

# Phát hiện và xử lý ngoại lệ
Q1 = data['Giá'].quantile(0.25)
Q3 = data['Giá'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Thay thế ngoại lệ bằng giá trị trung vị của biến
data['Giá'] = data['Giá'].apply(lambda x: x if lower_bound <= x <= upper_bound else data['Giá'].median())

# Tách biến đặc trưng và biến mục tiêu
X = data.drop(columns='Giá')
y = data['Giá']

# Định nghĩa cột dữ liệu số và phân loại
numerical_cols = ['Số km đã đi']
categorical_cols = ['Năm sản xuất', 'Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']

# Tiền xử lý cho dữ liệu số
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Tiền xử lý cho dữ liệu phân loại
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Gộp tiền xử lý cho dữ liệu số và phân loại
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Định nghĩa mô hình
model = RandomForestRegressor(random_state=10)

# Tạo pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, random_state=39)

# Huấn luyện mô hình
pipeline.fit(X_train, y_train)

# Dự đoán
y_pred = pipeline.predict(X_test)

# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 92347193.88918588
Mean Squared Error (MSE): 3.413418116241361e+16
Root Mean Squared Error (RMSE): 184754380.63118723
R^2 Score: 0.5675563022529857


Đối với phương pháp bỏ ngoại lệ, sẽ loại bỏ các điểm dữ liệu được coi là ngoại lệ khỏi tập dữ liệu trước khi huấn luyện mô hình. Dưới đây là so sánh giữa phương pháp thay thế ngoại lệ và phương pháp bỏ ngoại lệ:

### Phương pháp thay thế ngoại lệ:
- Thế mạnh:
  - Dữ liệu không bị mất đi hoàn toàn, mà chỉ bị thay thế bằng giá trị khác.
  - Có thể áp dụng cho các biến có nhiều ngoại lệ mà không cần loại bỏ dữ liệu.
- Hạn chế:
  - Có thể làm biến dạng phân phối của biến đó, ảnh hưởng đến hiệu suất của mô hình.

### Phương pháp bỏ ngoại lệ:
- Thế mạnh:
  - Loại bỏ ngoại lệ khỏi tập dữ liệu có thể làm giảm ảnh hưởng của chúng đối với hiệu suất của mô hình.
  - Giảm nhiễu trong dữ liệu, cải thiện khả năng dự đoán của mô hình.
- Hạn chế:
  - Có thể dẫn đến mất mát thông tin quan trọng nếu các điểm bị loại bỏ không phải là ngoại lệ thực sự.
  - Có thể làm giảm kích thước tập dữ liệu huấn luyện.

### So sánh:
Dựa trên kết quả bạn đã cung cấp, phương pháp thay thế ngoại lệ đã tạo ra một kết quả tốt hơn so với phương pháp bỏ ngoại lệ. MAE, MSE và RMSE của phương pháp thay thế ngoại lệ đều nhỏ hơn, và R² Score cao hơn so với phương pháp bỏ ngoại lệ. Điều này cho thấy rằng phương pháp thay thế ngoại lệ có khả năng sử dụtrường hợp của bạn. Tuy nhiên, việc lựa chọn phương pháp phụ thuộc vào đặc điểm cụ thể của dữ liệu và mục tiêu của mô hình.

In [125]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu
data = pd.read_csv("data_processed1.csv")

# Phát hiện và xử lý ngoại lệ
Q1 = data['Giá'].quantile(0.3)
Q3 = data['Giá'].quantile(0.66)
IQR = Q3 - Q1
lower_bound = Q1 - 1.0 * IQR
upper_bound = Q3 + 1.0 * IQR

# Thay thế ngoại lệ bằng giá trị trung vị của biến
data['Giá'] = data['Giá'].apply(lambda x: x if lower_bound <= x <= upper_bound else data['Giá'].median())

# Tách biến đặc trưng và biến mục tiêu
X = data.drop(columns='Giá')
y = data['Giá']

# Định nghĩa cột dữ liệu số và phân loại
numerical_cols = ['Số km đã đi']
categorical_cols = ['Năm sản xuất', 'Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']

# Tiền xử lý cho dữ liệu số
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Tiền xử lý cho dữ liệu phân loại
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Gộp tiền xử lý cho dữ liệu số và phân loại
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Định nghĩa mô hình
model = RandomForestRegressor(random_state=10)

# Tạo pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, random_state=39)

# Huấn luyện mô hình
pipeline.fit(X_train, y_train)

# Dự đoán
y_pred = pipeline.predict(X_test)

# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 41109833.78207731
Mean Squared Error (MSE): 6996948948130698.0
Root Mean Squared Error (RMSE): 83647767.14372416
R^2 Score: 0.7252235834686278


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu
data = pd.read_csv("data_processed1.csv")

# Phát hiện và xử lý ngoại lệ
Q1 = data['Giá'].quantile(0.3)
Q3 = data['Giá'].quantile(0.66)
IQR = Q3 - Q1
lower_bound = Q1 - 1.0 * IQR
upper_bound = Q3 + 1.0 * IQR

# Thay thế ngoại lệ bằng giá trị trung vị của biến
data['Giá'] = data['Giá'].apply(lambda x: x if lower_bound <= x <= upper_bound else data['Giá'].median())

# Tách biến đặc trưng và biến mục tiêu
X = data.drop(columns='Giá')
y = data['Giá']

# Định nghĩa cột dữ liệu số và phân loại
numerical_cols = ['Số km đã đi']
categorical_cols = ['Năm sản xuất', 'Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']

# Tiền xử lý cho dữ liệu số
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Tiền xử lý cho dữ liệu phân loại
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Gộp tiền xử lý cho dữ liệu số và phân loại
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Định nghĩa mô hình
model = RandomForestRegressor(random_state=10)

# Tạo pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, random_state=39)

# Huấn luyện mô hình
pipeline.fit(X_train, y_train)

# Dự đoán
y_pred = pipeline.predict(X_test)

# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 41109833.78207731
Mean Squared Error (MSE): 6996948948130698.0
Root Mean Squared Error (RMSE): 83647767.14372416
R^2 Score: 0.7252235834686278


In [3]:
# Đọc dữ liệu
data = pd.read_csv("data_processed1.csv")

# Lấy các giá trị duy nhất của các cột phân loại
unique_values = {
    'Kiểu dáng': data['Kiểu dáng'].unique(),
    'Xuất xứ': data['Xuất xứ'].unique(),
    'Hộp số': data['Hộp số'].unique(),
    'Nhiên liệu': data['Nhiên liệu'].unique()
}

# Hiển thị các giá trị duy nhất
for col, values in unique_values.items():
    print(f"{col}: {values}")


Kiểu dáng: ['Sedan' 'SUV' 'MPV' 'Pick-upTruck' 'Hatchback' 'Convertible' 'CUV'
 'Coupe' 'Van/Minivan' 'SportCar' 'Truck']
Xuất xứ: ['Trongnước' 'Nhậpkhẩu']
Hộp số: ['Sốtựđộng' 'Sốtay' 'Sốhỗnhợp']
Nhiên liệu: ['Xăng' 'Diesel' 'Loạikhác']


In [12]:
import pandas as pd

# Dữ liệu đầu vào mới
new_data = pd.DataFrame({
    'Số km đã đi': [5000],
    'Năm sản xuất': [2023],
    'Kiểu dáng': ['SUV'],
    'Xuất xứ': ['Nhậpkhẩu'],
    'Hộp số': ['sốhỗnhợp'],
    'Nhiên liệu': ['Xăng']
})

# Dự đoán giá
predicted_price = pipeline.predict(new_data)

# print(f"Giá dự đoán: {predicted_price[0]}")

formatted_price = format(predicted_price[0], ",.0f")  # Định dạng số với dấu "," phân cách hàng nghìn
print("Giá xe dự đoán:", formatted_price,"VND")


Giá xe dự đoán: 563,640,000 VND


In [10]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Định nghĩa pipeline như đã làm trước đây
categorical_cols = ['Năm sản xuất', 'Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']

# Truy xuất OneHotEncoder từ pipeline
encoder = pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']

# Truy xuất các giá trị phân loại đã được huấn luyện
encoded_categories = encoder.categories_

# Tạo một dictionary để hiển thị các giá trị phân loại đã được huấn luyện
category_dict = {categorical_cols[i]: encoded_categories[i] for i in range(len(categorical_cols))}

# Hiển thị các giá trị phân loại đã được huấn luyện
for col, values in category_dict.items():
    print(f"{col}: {values}")


Năm sản xuất: [1987 1993 1995 1996 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011
 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022]
Kiểu dáng: ['CUV' 'Convertible' 'Coupe' 'Hatchback' 'MPV' 'Pick-upTruck' 'SUV'
 'Sedan' 'SportCar' 'Truck' 'Van/Minivan']
Xuất xứ: ['Nhậpkhẩu' 'Trongnước']
Hộp số: ['Sốhỗnhợp' 'Sốtay' 'Sốtựđộng']
Nhiên liệu: ['Diesel' 'Loạikhác' 'Xăng']


# đổi phần này

# Thêm 'Năm sản xuất' vào các cột số
numerical_cols = ['Số km đã đi', 'Năm sản xuất']
categorical_cols = ['Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu
data = pd.read_csv("data_processed1.csv")

# Phát hiện và xử lý ngoại lệ
Q1 = data['Giá'].quantile(0.3)
Q3 = data['Giá'].quantile(0.66)
IQR = Q3 - Q1
lower_bound = Q1 - 1.0 * IQR
upper_bound = Q3 + 1.0 * IQR

# Thay thế ngoại lệ bằng giá trị trung vị của biến
data['Giá'] = data['Giá'].apply(lambda x: x if lower_bound <= x <= upper_bound else data['Giá'].median())

# Tách biến đặc trưng và biến mục tiêu
X = data.drop(columns='Giá')
y = data['Giá']

# Thêm 'Năm sản xuất' vào các cột số
numerical_cols = ['Số km đã đi', 'Năm sản xuất']
categorical_cols = ['Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']


# Tiền xử lý cho dữ liệu số
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Tiền xử lý cho dữ liệu phân loại
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# Gộp tiền xử lý cho dữ liệu số và phân loại
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])




# Định nghĩa mô hình
model = RandomForestRegressor(random_state=10)

# Tạo pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.14, random_state=39)

# Huấn luyện mô hình
pipeline.fit(X_train, y_train)

# Dự đoán
y_pred = pipeline.predict(X_test)

# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 38039357.18157267
Mean Squared Error (MSE): 6286758333668131.0
Root Mean Squared Error (RMSE): 79289080.79722032
R^2 Score: 0.753113401379672


In [124]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu
data = pd.read_csv("data_processed1.csv")

# Phát hiện và xử lý ngoại lệ
Q1 = data['Giá'].quantile(0.3)
Q3 = data['Giá'].quantile(0.66)
IQR = Q3 - Q1
lower_bound = Q1 - 1.0 * IQR
upper_bound = Q3 + 1.0 * IQR

# Thay thế ngoại lệ bằng giá trị trung vị của biến
data['Giá'] = data['Giá'].apply(lambda x: x if lower_bound <= x <= upper_bound else data['Giá'].median())

# Tách biến đặc trưng và biến mục tiêu
X = data.drop(columns='Giá')
y = data['Giá']

# Thêm 'Năm sản xuất' vào các cột số
numerical_cols = ['Số km đã đi', 'Năm sản xuất']
categorical_cols = ['Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']


# Tiền xử lý cho dữ liệu số
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Tiền xử lý cho dữ liệu phân loại
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# Gộp tiền xử lý cho dữ liệu số và phân loại
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])




# Định nghĩa mô hình
model = RandomForestRegressor(random_state=10)

# Tạo pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2   , random_state=69          )

# Huấn luyện mô hình
pipeline.fit(X_train, y_train)

# Dự đoán
y_pred = pipeline.predict(X_test)

# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 39737009.59844129
Mean Squared Error (MSE): 5813762927818705.0
Root Mean Squared Error (RMSE): 76248035.5669489
R^2 Score: 0.7646486741568997


In [172]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu
data = pd.read_csv("data_processed1.csv")

# Phát hiện và xử lý ngoại lệ
Q1 = data['Giá'].quantile(0.3)
Q3 = data['Giá'].quantile(0.66)
IQR = Q3 - Q1
lower_bound = Q1 - 1.0 * IQR
upper_bound = Q3 + 1.0 * IQR

# Thay thế ngoại lệ bằng giá trị trung vị của biến
data['Giá'] = data['Giá'].apply(lambda x: x if lower_bound <= x <= upper_bound else data['Giá'].median())

# Tách biến đặc trưng và biến mục tiêu
X = data.drop(columns='Giá')
y = data['Giá']

# Thêm 'Năm sản xuất' vào các cột số
numerical_cols = ['Số km đã đi', 'Năm sản xuất']
categorical_cols = ['Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']


# Tiền xử lý cho dữ liệu số
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Tiền xử lý cho dữ liệu phân loại
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# Gộp tiền xử lý cho dữ liệu số và phân loại
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])




# Định nghĩa mô hình
model = RandomForestRegressor(random_state=10)

# Tạo pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18   , random_state=74          )

# Huấn luyện mô hình
pipeline.fit(X_train, y_train)

# Dự đoán
y_pred = pipeline.predict(X_test)

# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 35205601.97997367
Mean Squared Error (MSE): 5012718466726283.0
Root Mean Squared Error (RMSE): 70800554.14137861
R^2 Score: 0.7804876635059604


In [212]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu
data = pd.read_csv("data_processed1.csv")

# Phát hiện và xử lý ngoại lệ
Q1 = data['Giá'].quantile(0.3)
Q3 = data['Giá'].quantile(0.66)
IQR = Q3 - Q1
lower_bound = Q1 - 1.2 * IQR
upper_bound = Q3 + 0.9 * IQR

# Thay thế ngoại lệ bằng giá trị trung vị của biến
data['Giá'] = data['Giá'].apply(lambda x: x if lower_bound <= x <= upper_bound else data['Giá'].median())

# Tách biến đặc trưng và biến mục tiêu
X = data.drop(columns='Giá')
y = data['Giá']

# Thêm 'Năm sản xuất' vào các cột số
numerical_cols = ['Số km đã đi', 'Năm sản xuất']
categorical_cols = ['Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']


# Tiền xử lý cho dữ liệu số
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Tiền xử lý cho dữ liệu phân loại
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# Gộp tiền xử lý cho dữ liệu số và phân loại
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])




# Định nghĩa mô hình
model = RandomForestRegressor(random_state=10)

# Tạo pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18   , random_state=74          )

# Huấn luyện mô hình
pipeline.fit(X_train, y_train)

# Dự đoán
y_pred = pipeline.predict(X_test)

# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 33752986.290213905
Mean Squared Error (MSE): 4701517681893594.0
Root Mean Squared Error (RMSE): 68567613.94341788
R^2 Score: 0.8043249723678296


In [271]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu
data = pd.read_csv("data_processed1.csv")

# Phát hiện và xử lý ngoại lệ
Q1 = data['Giá'].quantile(0.3)
Q3 = data['Giá'].quantile(0.66)
IQR = Q3 - Q1
lower_bound = Q1 - 1.9 * IQR
upper_bound = Q3 + 0.9 * IQR

# Thay thế ngoại lệ bằng giá trị trung vị của biến
data['Giá'] = data['Giá'].apply(lambda x: x if lower_bound <= x <= upper_bound else data['Giá'].median())

# Tách biến đặc trưng và biến mục tiêu
X = data.drop(columns='Giá')
y = data['Giá']

# Thêm 'Năm sản xuất' vào các cột số
numerical_cols = ['Số km đã đi', 'Năm sản xuất']
categorical_cols = ['Kiểu dáng', 'Xuất xứ', 'Hộp số', 'Nhiên liệu']


# Tiền xử lý cho dữ liệu số
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Tiền xử lý cho dữ liệu phân loại
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# Gộp tiền xử lý cho dữ liệu số và phân loại
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])




# Định nghĩa mô hình
model = RandomForestRegressor(random_state=10)

# Tạo pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.18   , random_state=74          )

# Huấn luyện mô hình
pipeline.fit(X_train, y_train)

# Dự đoán
y_pred = pipeline.predict(X_test)

# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R^2 Score: {r2}")


Mean Absolute Error (MAE): 33858708.52017291
Mean Squared Error (MSE): 4645004264212674.0
Root Mean Squared Error (RMSE): 68154268.12909573
R^2 Score: 0.8654045576365712
